<a href="https://colab.research.google.com/github/GavinButts/Fall-Data-Challenge-2022/blob/main/Code/Kevin/Ordinal%20-%20SEGRADES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
!pip install --upgrade --no-deps statsmodels
import scipy.stats as stats


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 38.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [2]:
sheet_id = "11rno0SjsOyV1JlevH3q40UMm1s5e6m7g"
sheet_name = "curated_2019-required"

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

df = pd.read_csv(url)


In [3]:
#create FOYESCNT variable that determines the amount of times each person said 'Yes' on the FO#### questions (ITEMS 72-74)
#Legend:  Number of 'Yes'

FOYESCNT = []      #variable that will be added to csv
yesCount = 0
for i in range(15500):
  if df['FOSTORY2X'][i] == 1:
    yesCount += 1
  if df['FOCRAFTS'][i] == 1:
    yesCount += 1
  if df['FOGAMES'][i] == 1:
    yesCount += 1
  if df['FOBUILDX'][i] == 1:
    yesCount += 1
  if df['FOSPORT'][i] == 1:
    yesCount += 1
  if df['FORESPON'][i] == 1:
    yesCount += 1
  if df['FOHISTX'][i] == 1:
    yesCount += 1
  if df['FOLIBRAYX'][i] == 1:
    yesCount += 1
  if df['FOBOOKSTX'][i] == 1:
    yesCount += 1
  if df['FODINNERX'][i] != 0:
    yesCount += 1
  FOYESCNT.append(yesCount)
  yesCount = 0

df["FOYESCNT"] = FOYESCNT
df.to_csv("sample.csv", index=False)  #adds variable to csv

#create FSYESCNT variable that determines the amount of times each person said 'Yes' on the FS#### questions (ITEMS 60)
#Legend:  Number of 'Yes'

FSYESCNT = []      #variable that will be added to csv
yesCount = 0
FSYESCNT_LMH = []

for i in range(15500):
  if df['FSSPORTX'][i] == 1:
    yesCount += 1
  if df['FSVOL'][i] == 1:
    yesCount += 1
  if df['FSMTNG'][i] == 1:
    yesCount += 1
  if df['FSPTMTNG'][i] == 1:
    yesCount += 1
  if df['FSATCNFN'][i] == 1:
    yesCount += 1
  if df['FSFUNDRS'][i] == 1:
    yesCount += 1
  if df['FSCOMMTE'][i] == 1:
    yesCount += 1
  if df['FSCOUNSLR'][i] == 1:
    yesCount += 1
  FSYESCNT.append(yesCount)
  if yesCount <= 3:
    FSYESCNT_LMH.append('1) LOW INVOLVMENT')
  elif yesCount > 3 and yesCount<=5:
    FSYESCNT_LMH.append('2) MEDIUM INVOLVEMENT')
  elif yesCount > 5:
    FSYESCNT_LMH.append('3) HIGH INVOLVEMENT')
  yesCount = 0

df["FSYESCNT"] = FSYESCNT
df["FSYESCNT_LMH"] = FSYESCNT_LMH

df.to_csv("sample.csv", index=False)  #adds variable to csv


In [4]:
#Data Pre-Processing
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[-1,1,2,3,4,5], ordered=True)
df["SEGRADES"] = df["SEGRADES"].astype(cat_type)
df["SEGRADES"].dtype

CategoricalDtype(categories=[-1, 1, 2, 3, 4, 5], ordered=True)

In [5]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [13]:
from os import XATTR_CREATE
#['HHPARN19X','FHPLACE','SEABSNT','FSYESCNT','SEGRADES','FOYESCNT','RACEETH','TTLHHINC']
x = ['ALLGRADEX','EDCPUB','SEGRADEQ','FHPLACE',
     'SEABSNT','RACEETH','OWNRNTHB','SEFUTUREX']
mod_prob = OrderedModel(df['SEGRADES'],
                        df[x],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()


Optimization terminated successfully.
         Current function value: 1.028578
         Iterations: 44
         Function evaluations: 47
         Gradient evaluations: 47


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:               SEGRADES   Log-Likelihood:                -15943.
Model:                   OrderedModel   AIC:                         3.191e+04
Method:            Maximum Likelihood   BIC:                         3.201e+04
Date:                Fri, 28 Oct 2022                                         
Time:                        17:22:46                                         
No. Observations:               15500                                         
Df Residuals:                   15487                                         
Df Model:                          13                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ALLGRADEX     -0.1435      0.005    -31.297      0.000      -0.152      -0.134
EDCPUB        -0.1508      0.055     -2.717      0.007      -0.260      -0.042
SEGRADEQ       1.3201      0.022     60.044      0.000       1.277       1.363
FHPLACE       -0.1776      0.025     -7.202      0.000      -0.226      -0.129
SEABSNT        0.2132      0.028      7.506      0.000       0.158       0.269
RACEETH        0.0383      0.013      2.942      0.003       0.013       0.064
OWNRNTHB       0.1196      0.034      3.510      0.000       0.053       0.186
SEFUTUREX     -0.1267      0.014     -8.827      0.000      -0.155      -0.099
-1/1          -8.9484      0.719    -12.454      0.000     -10.357      -7.540
1/2            2.2762      0.072     31.408      0.000       2.134       2.418
2/3            0.6126      0.014     43.935      0.000       0.585       0.640
3/4           -0.2979      0.026    -11.274      0.000      -0.350      -0.246
4/5           -1.8329      0.063    -29.153      0.000      -1.956      -1.710
==============================================================================
"""

In [14]:
predicted = res_log.model.predict(res_log.params, exog=df[x])
predicted


/usr/local/lib/python3.7/dist-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


array([[3.14419441e-04, 8.41885878e-01, 1.29046644e-01, 1.48579501e-02,
        2.02958639e-03, 1.18655222e-02],
       [7.39920445e-06, 1.11546003e-01, 3.31241705e-01, 1.82612090e-01,
        3.66598735e-02, 3.37932929e-01],
       [5.74282371e-06, 8.87927616e-02, 2.92686713e-01, 1.82939719e-01,
        3.88420471e-02, 3.96733017e-01],
       ...,
       [8.72823191e-05, 5.96889532e-01, 3.06636151e-01, 4.80686622e-02,
        6.84615211e-03, 4.14722202e-02],
       [4.07124323e-04, 8.73193281e-01, 1.04049477e-01, 1.15859426e-02,
        1.57652801e-03, 9.18764660e-03],
       [3.33020452e-05, 3.61042271e-01, 4.20427367e-01, 1.01050467e-01,
        1.55925405e-02, 1.01854052e-01]])

In [16]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((np.asarray(df['SEGRADES'].values.codes) == pred_choice).mean())


Fraction of correct choice predictions
0.594
